In [ ]:
file_name = "IPB-20200226.docx" #ダウンロードされるファイル名
mindate='2019-06-27' #これより後の業績を集める
maxdate='2020-02-27' #これより前の業績を集める
smark='' #謝辞に課題番号ありの論文にマーク付ける場合はここで指定。中間報告では'▲'。
allenglish = False #名前表記をすべて英語で統一する場合はTrue, 論文以外の名前表記を日本語にする場合False

In [ ]:
%pip install python-docx
import requests,json,sys,os, gspread, time
import numpy as np
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_UNDERLINE
from google.colab import files,auth
from oauth2client.client import GoogleCredentials
# 認証処理
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1wce1XHSFGSBttupnSIqe_5abtijBb_hBYM2bfaV9Jn4/edit#gid=0')

In [ ]:
worksheet = sh.get_worksheet(0)
fullList = worksheet.get_all_values()
surnameind=fullList[0].index('Surname')
firstnameind=fullList[0].index('First name')
surnameindJP=fullList[0].index('苗字')
firstnameindJP=fullList[0].index('名')
rIDind=fullList[0].index('researchmapID')
gIDind=fullList[0].index('grantID')
dbind=fullList[0].index('代表分担協力')
membernum=len(fullList)-1

allnames=[fullList[i+1][surnameind] + ' ' + fullList[i+1][firstnameind] for i in range(membernum)]
allSurname=[fullList[i+1][surnameind] for i in range(membernum)]
allnamesJP=[fullList[i+1][surnameindJP] + ' ' + fullList[i+1][firstnameindJP] for i in range(membernum)]
allmembers=[fullList[i+1][rIDind] for i in range(membernum)]
allDaihyoBuntan=[fullList[i+1][dbind] for i in range(membernum)]
grant_numbers=[fullList[i+1][gIDind] for i in range(membernum)]

#Exception handling
nameList=allnames+['Sagawa T', 'Tetsuya J kobayashi']
daihyobuntanList=allDaihyoBuntan+['D','D']

In [ ]:
# Function to set the name order (surname-first name)
def SurnameFirst(namesDic,sn):
    oldnamelist=[]
    swap=0
    for indiv in namesDic:
        oldnamelist=oldnamelist+[indiv['name'].replace(',','').replace('.','')]
        #print(oldnamelist)
    for name in oldnamelist:
        if sn in name.split(' '):
            if name.split(' ').index(sn)==0:
                swap=0
                break;
            else:
                swap=1
                break;
    if swap:
        newnamelist=[]
        for name in oldnamelist:
            namesplit=name.split(' ')
            names=[namesplit[-1]]+namesplit[:-1]
            newnamelist=newnamelist+[' '.join(names)]
    else:
        newnamelist=oldnamelist
    return newnamelist
  

In [ ]:
#download json files from researchmap
url = "https://api.researchmap.jp/"
itemslist = ["published_papers","research_projects","misc","presentations","books_etc"]
jsonfiles={}
for name in allmembers:
  jsonfiles[name]={}
      for it in itemslist:
          r1 = requests.get(url+name+'/'+it)
          jsonfiles[name][it]=json.loads(r1.text)

In [ ]:
# make dictionary of all papers
i=0
PapersDict={}

doilist=[]
doiDict={}
#dc = Document()
for ids,fullname,dh in zip(allmembers,allnames,allDaihyoBuntan):
    surname=fullname.split(' ')[0]
    dfP = jsonfiles[ids]["published_papers"]
    dfG = jsonfiles[ids]["research_projects"]
    if 'items' in dfG.keys():
        grantID="0"
        for dfs in dfG['items']:
            if 'identifiers' in dfs.keys():
                if 'grant_number' in dfs['identifiers'].keys():
                    if dfs['identifiers']['grant_number'][0] in grant_numbers:
                        grantID=dfs['rm:id']
                        break
    if 'items' in dfP.keys():    
        for dfs in dfP['items']:
            if "authors" not in dfs.keys():
                continue
            if 'identifiers' in dfs.keys():
                doinum=[0]
                if 'doi' in dfs['identifiers'].keys():
                    doinum=dfs['identifiers']['doi']
                    #print(doinum)

                PapersDict[i]={}
                correspo=False
                Ryoiki=False
                if 'research_project_id' in dfs['identifiers'].keys():
                    if grantID in dfs['identifiers']['research_project_id']:
                        Ryoiki=True
                if "published_paper_owner_roles" in dfs.keys():
                    if "corresponding" in dfs["published_paper_owner_roles"]:
                        correspo=True
                        
                if "publication_name" in dfs.keys():
                    if 'en' in dfs["publication_name"].keys():
                        jname=dfs["publication_name"]['en'].upper()
                        Sname=SurnameFirst(dfs["authors"]['en'],surname)
                    else:
                        jname=dfs["publication_name"]['ja'].upper()
                        Sname=SurnameFirst(dfs["authors"]['ja'],surname)
                else:
                    if 'en' in dfs["paper_title"].keys():
                        jname=dfs["paper_title"]['en'].upper()
                        Sname=SurnameFirst(dfs["authors"]['en'],surname)
                    else:
                        jname=dfs["paper_title"]['ja'].upper()
                        Sname=SurnameFirst(dfs["authors"]['ja'],surname)
                spage=''  
                if "starting_page" in dfs.keys():
                    spage=', '+dfs["starting_page"]
                vol=''
                if "volume" in dfs.keys():
                    vol=dfs["volume"]
                if doinum in doilist:
                    print('aaaa')
                    doiDict[doinum[0]]['name']=doiDict[doinum[0]]['name']+[fullname]
                    doiDict[doinum[0]]['Corresp']=doiDict[doinum[0]]['Corresp']+[correspo]
                else:
                    doiDict[doinum[0]]={}
                    doiDict[doinum[0]]['name']=[fullname]
                    doiDict[doinum[0]]['Corresp']=[correspo]
                    doilist=doilist+[doinum[0]]
                text1="\""+dfs['paper_title']['en']+"\"" +', '
                text2=jname+', '+vol+spage+ ' ('+dfs["publication_date"][:4] +').'
                PapersDict[i]['text1']=text1
                PapersDict[i]['text2']=text2
                PapersDict[i]['researcher']=fullname
                PapersDict[i]['authors']=Sname
                PapersDict[i]['date']=dfs["publication_date"]
                PapersDict[i]['referee']=dfs['referee']
                PapersDict[i]['doi']=doinum[0]
                PapersDict[i]['ryoiki']=Ryoiki
                PapersDict[i]['Daihyo']=dh
                PapersDict[i]['Corresp']=correspo
                i=i+1

In [ ]:
# make dictionary of all talks
TalksDict={}
i=0
for ids,fullname,fullnameJP,dh in zip(allmembers,allnames,allnamesJP,allDaihyoBuntan):
    dfPr = jsonfiles[ids]["presentations"]
    if 'items' in dfPr.keys():
        for dfs in dfPr['items']:
            if all([a in dfs.keys() for a in ['invited',"presentation_title","event",'publication_date',"presenters"]]):
                if dfs['invited']:
                    if ('ja' in dfs["presenters"].keys()):
                        pname=dfs["presenters"]["ja"][0]["name"]
                    else:
                        pname=dfs["presenters"]["en"][0]["name"]
                    if ('ja' in dfs["event"].keys()):
                        ename=dfs["event"]["ja"]
                    else:
                        ename=dfs["event"]["en"]
                    if ('ja' in dfs["presentation_title"].keys()):
                        ptitle=dfs["presentation_title"]["ja"]
                    else:
                        ptitle=dfs["presentation_title"]["en"]
                    pdate=dfs["publication_date"]
                    TalksDict[i]={}
                    TalksDict[i]["presenter"]=fullname
                    if allenglish:
                      TalksDict[i]['printname']=fullname
                    else:
                      TalksDict[i]["printname"]=fullnameJP
                    TalksDict[i]["event"]=ename
                    TalksDict[i]["presentation_title"]=ptitle
                    TalksDict[i]["date"]=pdate
                    i=i+1
# %%

In [ ]:
# make dictionary of all books_etc
booksDict={}
i=0
for ids,fullname,fullnameJP,dh in zip(allmembers,allnames,allnamesJP,allDaihyoBuntan):
  dfM = jsonfiles[ids]["books_etc"]
  if 'items' in dfM.keys():
    for dfs in dfM['items']:
      if all([a in dfs.keys() for a in ['authors',"book_title","publication_date"]]):
        if ('ja' in dfs["authors"].keys()):
            pname=dfs["authors"]["ja"][0]["name"]
        else:
            pname=dfs["authors"]["en"][0]["name"]
        if ('ja' in dfs["book_title"].keys()):
            ename=dfs["book_title"]["ja"]
        else:
            ename=dfs["book_title"]["en"]
        if "book_owner_range" in dfs.keys():
          if ('ja' in dfs["book_owner_range"].keys()):
            eoname=" \'"+dfs["book_owner_range"]["ja"]+"\',"
          else:
            eoname=" \'"+dfs["book_owner_range"]["en"]+"\',"
        else:
          eoname=''
        if "book_owner_role" in dfs.keys():
          brole=" ("+dfs["book_owner_role"]+"),"
        else:
          brole=','
        if "publisher" in dfs.keys():
          if ('ja' in dfs["publisher"].keys()):
            pub=" "+dfs["publisher"]["ja"]+","
          else:
            pub=" "+dfs["publisher"]["en"]+","
        else:
          pub=''
        pdate=dfs["publication_date"]
        #print(pname,ename,pdate)
        booksDict[i]={}
        booksDict[i]['authors']=fullname
        if allenglish:
          booksDict[i]['printname']=fullname
        else:
          booksDict[i]['printname']=fullnameJP
        booksDict[i]["book_title"]=' \"'+ename+'\",'
        booksDict[i]["book_owner_role"]=brole
        booksDict[i]["book_owner_range"]=eoname
        booksDict[i]["publisher"]=pub
        booksDict[i]["date"]=pdate
        i=i+1

In [ ]:
# make dictionary of all MISCs
miscDict={}
i=0
for ids,fullname,fullnameJP,dh in zip(allmembers,allnames,allnamesJP,allDaihyoBuntan):
  dfM = jsonfiles[ids]["misc"]
  if 'items' in dfM.keys():
    for dfs in dfM['items']:
      if all([a in dfs.keys() for a in ['authors',"paper_title","publication_date","publication_name"]]):
        if ('ja' in dfs["authors"].keys()):
            pname=dfs["authors"]["ja"][0]["name"]
        else:
            pname=dfs["authors"]["en"][0]["name"]
        if ('ja' in dfs["paper_title"].keys()):
            ename=dfs["paper_title"]["ja"]
        else:
            ename=dfs["paper_title"]["en"]
        if ('ja' in dfs["publication_name"].keys()):
            ptitle=dfs["publication_name"]["ja"]
        else:
            ptitle=dfs["publication_name"]["en"]
        pdate=dfs["publication_date"]
        #print(pname,ename,pdate)
        miscDict[i]={}
        miscDict[i]['authors']=fullname
        if allenglish:
          miscDict[i]['printname']=fullname
        else:
          miscDict[i]['printname']=fullnameJP

        miscDict[i]["paper_title"]=' \''+ename+'\','
        miscDict[i]["publication_name"]=' \"'+ptitle+'\",'
        miscDict[i]["date"]=pdate
        i=i+1

In [ ]:
# generate docx

PapersDictSelected={k:PapersDict[k] for k in range(len(PapersDict)) if (PapersDict[k]['date']>mindate) & (PapersDict[k]['date']<maxdate)  & (PapersDict[k]['referee'])}

keys=list(PapersDictSelected.keys())
datelist=[PapersDictSelected[r]['date'] for r in keys]
arg=np.argsort(datelist)[::-1]

document = Document()
document.add_paragraph('原著論文')
inds=0
for r in arg:
    inds=inds+1
    pap=PapersDictSelected[keys[r]]
#     if len(doiDict[pap['doi']]['name'])>1:
#         print(doiDict[pap['doi']]['name'])
#         continue;
    if pap['ryoiki']:
        p = document.add_paragraph(smark+str(inds)+'. '+pap['text1'])
    else:
        p = document.add_paragraph(str(inds)+'. '+pap['text1'])
    for nm in pap['authors']:
        if nm in nameList:
            if pap['Corresp']:
                p.add_run('*')
            if daihyobuntanList[nameList.index(nm)]=='D':
                p.add_run(nm).underline = WD_UNDERLINE.DOUBLE
            elif daihyobuntanList[nameList.index(nm)]=='B':
                p.add_run(nm).underline = True
            else:
                p.add_run(nm)
        else:
            p.add_run(nm)
        p.add_run(', ')
    p.add_run(pap['text2'])
    
TalksDictSelected={k:TalksDict[k] for k in range(len(TalksDict)) if (TalksDict[k]['date']>mindate) & (TalksDict[k]['date']<maxdate) }

keys=list(TalksDictSelected.keys())
datelist=[TalksDictSelected[r]['date'] for r in keys]
arg=np.argsort(datelist)[::-1]
document.add_paragraph('')
document.add_paragraph('招待講演')
inds=0
for r in arg:
    inds=inds+1
    pap=TalksDictSelected[keys[r]]
    p = document.add_paragraph(str(inds)+'. ')
    nm=pap["presenter"]
    if daihyobuntanList[nameList.index(nm)]=='D':
        p.add_run(pap["printname"]).underline = WD_UNDERLINE.DOUBLE
    elif daihyobuntanList[nameList.index(nm)]=='B':
        p.add_run(pap["printname"]).underline = True
    else:
        p.add_run(pap["printname"])
    p.add_run(', \"'+pap["presentation_title"]+"\"")
    p.add_run(', '+pap["event"])
    p.add_run(', '+pap["date"]+'.')

booksDictSelected={k:booksDict[k] for k in range(len(booksDict)) if (booksDict[k]['date']>mindate) & (booksDict[k]['date']<maxdate) }

keys=list(booksDictSelected.keys())
datelist=[booksDictSelected[r]['date'] for r in keys]
arg=np.argsort(datelist)[::-1]
document.add_paragraph('')
document.add_paragraph('書籍')
inds=0
for r in arg:
    inds=inds+1
    pap=booksDictSelected[keys[r]]
    p = document.add_paragraph(str(inds)+'. ')
    nm=pap['authors']
    if daihyobuntanList[nameList.index(nm)]=='D':
        p.add_run(pap["printname"]).underline = WD_UNDERLINE.DOUBLE
    elif daihyobuntanList[nameList.index(nm)]=='B':
        p.add_run(pap["printname"]).underline = True
    else:
        p.add_run(pap["printname"])
    p.add_run(pap["book_owner_role"])
    p.add_run(pap["book_owner_range"])
    p.add_run(pap["book_title"])
    p.add_run(pap["publisher"])
    p.add_run(' '+pap["date"][:7]+'.')
document.save(file_name)

miscDictSelected={k:miscDict[k] for k in range(len(miscDict)) if (miscDict[k]['date']>mindate) & (miscDict[k]['date']<maxdate) }

keys=list(miscDictSelected.keys())
datelist=[miscDictSelected[r]['date'] for r in keys]
arg=np.argsort(datelist)[::-1]
document.add_paragraph('')
document.add_paragraph('その他')
inds=0
for r in arg:
    inds=inds+1
    pap=miscDictSelected[keys[r]]
    p = document.add_paragraph(str(inds)+'. ')
    nm=pap['authors']
    if daihyobuntanList[nameList.index(nm)]=='D':
        p.add_run(pap["printname"]).underline = WD_UNDERLINE.DOUBLE
    elif daihyobuntanList[nameList.index(nm)]=='B':
        p.add_run(pap["printname"]).underline = True
    else:
        p.add_run(pap["printname"])
    p.add_run(','+pap["paper_title"])
    p.add_run(pap["publication_name"])
    p.add_run(' '+pap["date"]+'.')
document.save(file_name)

In [ ]:
files.download(file_name)